In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('merge and split data').getOrCreate()

In [ ]:
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql.types import *
from typing import Iterator
import pandas as pd
import torch

In [ ]:
train = spark.read.text("/content/drive/MyDrive/Big Data/Output/KGAT-final/registered_courses.txt")
train.show(5)

+--------------------+
|               value|
+--------------------+
|0 2345 1216 2207 ...|
|10 2552 1883 2716...|
|100 1461 3144 202...|
|1000 1584 1242 31...|
|10000 2665 1814 2...|
+--------------------+
only showing top 5 rows



In [ ]:
test = spark.read.text("/content/drive/MyDrive/Big Data/Output/KGAT-final/test_light.txt")
test.show(5)

+----------+
|     value|
+----------+
|     0 184|
|   10 2338|
|  100 1348|
| 1000 1786|
|10000 3116|
+----------+
only showing top 5 rows



In [ ]:
train_user_dict = dict()
n_entities = 16118

# Process each row in the DataFrame
for row in train.rdd.collect():
    tmp = row[0].strip()
    inter = [int(i) for i in tmp.split()]

    if len(inter) > 1:
        user_id, item_ids = inter[0], inter[1:]
        item_ids = list(set(item_ids))
        train_user_dict[user_id] = item_ids

train_user_dict = {
            k + n_entities: np.unique(v).astype(np.int32)
            for k, v in train_user_dict.items()
        }

In [ ]:
print(train_user_dict)

{16118: array([ 160, 1179, 1190, 1198, 1216, 1234, 1238, 1278, 1319, 1322, 1334,
       1474, 1605, 1647, 1656, 1882, 2138, 2207, 2273, 2277, 2302, 2345,
       2416, 2544, 2572, 2630, 2750, 2880, 2961, 3016, 3143], dtype=int32), 16128: array([  17,  207,  406,  432,  504, 1272, 1630, 1719, 1752, 1883, 1927,
       1965, 2024, 2117, 2171, 2552, 2631, 2635, 2716, 2759, 2765, 2769,
       2806, 2963], dtype=int32), 16218: array([ 380, 1461, 2024, 2171, 2626, 2701, 2829, 2944, 3144], dtype=int32), 17118: array([1193, 1242, 1334, 1412, 1584, 1648, 1823, 1863, 2038, 2413, 2445,
       2571, 2620, 2624, 2817, 3087, 3127], dtype=int32), 26118: array([  45,  164,  907, 1256, 1260, 1436, 1443, 1461, 1466, 1628, 1633,
       1658, 1758, 1814, 1854, 1909, 1920, 1980, 1981, 2011, 2082, 2139,
       2168, 2208, 2341, 2475, 2516, 2572, 2646, 2654, 2663, 2665, 2673,
       2679, 2689, 2764, 2783, 2805, 2807, 2821, 2846, 2905, 2913, 2945,
       2953, 2966, 2992, 3003, 3005, 3052, 3054, 3111, 3117], d

In [ ]:
test_user_dict = dict()
n_entities = 16118

# Process each row in the DataFrame
for row in test.rdd.collect():
    tmp = row[0].strip()
    inter = [int(i) for i in tmp.split()]

    if len(inter) > 1:
        user_id, item_ids = inter[0], inter[1:]
        item_ids = list(set(item_ids))
        test_user_dict[user_id] = item_ids

test_user_dict = {
            k + n_entities: np.unique(v).astype(np.int32)
            for k, v in test_user_dict.items()
        }

In [ ]:
test_user_dict

{16118: array([184], dtype=int32),
 16128: array([2338], dtype=int32),
 16218: array([1348], dtype=int32),
 17118: array([1786], dtype=int32),
 26118: array([3116], dtype=int32),
 26121: array([817], dtype=int32),
 26123: array([2670], dtype=int32),
 26124: array([2783], dtype=int32),
 26125: array([1010], dtype=int32),
 26126: array([1086], dtype=int32)}

### Start mapping

In [3]:
result_df = spark.read.option("header", "true").csv("/content/drive/MyDrive/Big Data/Output/KGAT-final/evaluation_results.csv")

result_df.show(5)


NameError: name 'spark' is not defined

In [1]:
mapping_df = spark.read.option("header", "true").csv("/content/drive/MyDrive/Big Data/Output/user_mapping.csv")
mapping_df.show(5)

NameError: name 'spark' is not defined

In [2]:
mapping_course = spark.read.option("header", "true").csv("/content/drive/MyDrive/Big Data/Output/course_mapping.csv")
mapping_course.show(5)

NameError: name 'spark' is not defined

In [ ]:
user_df = spark.read.json("/content/drive/MyDrive/Big Data/Input/user.json")
user_df.show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
course_df = spark.read.json("/content/drive/MyDrive/Big Data/Input/course.json")
course_df.show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------+--------+--------------------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Bước 1: Join result_df với mapping_df để lấy mapped_id từ original_user_id
merged_df = result_df.join(mapping_df, result_df.original_user_id == mapping_df.mapped_id, how="left")
print("\nSau khi join result_df và mapping_df:")
merged_df.show(5)


Sau khi join result_df và mapping_df:
+----------------+--------+--------------------+-----------+---------+
|original_user_id|gt_index|         top10_preds|original_id|mapped_id|
+----------------+--------+--------------------+-----------+---------+
|               0|      -1|[1190, 1198, 1322...|   U_100090|        0|
|              10|      -1|[1630, 2552, 2554...|   U_100378|       10|
|             100|      17|[380, 2023, 2626,...| U_10210528|      100|
|            1000|      -1|[1648, 1176, 1334...|  U_1142122|     1000|
|           10000|      -1|[2595, 2504, 2274...| U_12855879|    10000|
+----------------+--------+--------------------+-----------+---------+
only showing top 5 rows



In [ ]:
# Bước 2: Join kết quả với df3 để lấy user_name từ original_id
final_df = merged_df.join(user_df, merged_df.original_id == user_df.id, how="left")
final_df = final_df.select("original_user_id", "name", "top10_preds")
print("\nKết quả cuối cùng (mapping_id → user_name):")
final_df.show(5)

In [ ]:
schema = ArrayType(IntegerType())
final_df = final_df.withColumn("top10_preds", from_json(final_df["top10_preds"],schema))

In [ ]:
df_with_position = final_df.select(
    col("original_user_id"),
    col("name").alias("User Name"),
    posexplode(col("top10_preds")).alias("position", "mapping_id")
)
df_with_position.show(5)

In [3]:
# Join và sắp xếp lại theo thứ tự gốc
df_ordered = df_with_position.join(mapping_course, df_with_position.mapping_id == mapping_course.mapped_id, how="left") \
    .join(course_df, mapping_course.original_id == course_df.id, how="left") \
    .orderBy("original_user_id", "position")

NameError: name 'df_with_position' is not defined

In [ ]:
# Collect lại với thứ tự
ordered_result = df_ordered.groupBy("original_user_id", "User Name").agg(
  collect_list("name").alias("Course Name")
)
ordered_result.show(5, truncate=False)

### End of mapping

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


def _L2_loss_mean(x):
    return torch.mean(torch.sum(torch.pow(x, 2), dim=1, keepdim=False) / 2.0)


class Aggregator(nn.Module):

    def __init__(self, in_dim, out_dim, dropout, aggregator_type):
        super(Aggregator, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.dropout = dropout
        self.aggregator_type = aggregator_type

        self.message_dropout = nn.Dropout(dropout)
        self.activation = nn.LeakyReLU()

        if self.aggregator_type == "gcn":
            self.linear = nn.Linear(self.in_dim, self.out_dim)  # W in Equation (6)
            nn.init.xavier_uniform_(self.linear.weight)

        elif self.aggregator_type == "graphsage":
            self.linear = nn.Linear(self.in_dim * 2, self.out_dim)  # W in Equation (7)
            nn.init.xavier_uniform_(self.linear.weight)

        elif self.aggregator_type == "bi-interaction":
            self.linear1 = nn.Linear(self.in_dim, self.out_dim)  # W1 in Equation (8)
            self.linear2 = nn.Linear(self.in_dim, self.out_dim)  # W2 in Equation (8)
            nn.init.xavier_uniform_(self.linear1.weight)
            nn.init.xavier_uniform_(self.linear2.weight)

        else:
            raise NotImplementedError

    def forward(self, ego_embeddings, A_in):
        """
        ego_embeddings:  (n_users + n_entities, in_dim)
        A_in:            (n_users + n_entities, n_users + n_entities), torch.sparse.FloatTensor
        """
        # Equation (3)
        side_embeddings = torch.matmul(A_in, ego_embeddings)

        if self.aggregator_type == "gcn":
            # Equation (6) & (9)
            embeddings = ego_embeddings + side_embeddings
            embeddings = self.activation(self.linear(embeddings))

        elif self.aggregator_type == "graphsage":
            # Equation (7) & (9)
            embeddings = torch.cat([ego_embeddings, side_embeddings], dim=1)
            embeddings = self.activation(self.linear(embeddings))

        elif self.aggregator_type == "bi-interaction":
            # Equation (8) & (9)
            sum_embeddings = self.activation(
                self.linear1(ego_embeddings + side_embeddings)
            )
            bi_embeddings = self.activation(
                self.linear2(ego_embeddings * side_embeddings)
            )
            embeddings = bi_embeddings + sum_embeddings

        embeddings = self.message_dropout(embeddings)  # (n_users + n_entities, out_dim)
        return embeddings


class KGAT(nn.Module):

    def __init__(
        self,
        n_users,
        n_entities,
        n_relations,
        use_pretrain=0,
        embed_dim=64,
        relation_dim=64,
        aggregation_type="bi-interaction",
        conv_dim_list="[64, 32, 16]",
        mess_dropout="[0.1, 0.1, 0.1]",
        kg_l2loss_lambda=1e-5,
        cf_l2loss_lambda=1e-5,
        A_in=None,
        user_pre_embed=None,
        item_pre_embed=None,
    ):

        super(KGAT, self).__init__()
        self.use_pretrain = use_pretrain

        self.n_users = n_users
        self.n_entities = n_entities
        self.n_relations = n_relations

        self.embed_dim = embed_dim
        self.relation_dim = relation_dim

        self.aggregation_type = aggregation_type
        self.conv_dim_list = [embed_dim] + eval(conv_dim_list)
        self.mess_dropout = eval(mess_dropout)
        self.n_layers = len(eval(conv_dim_list))

        self.kg_l2loss_lambda = kg_l2loss_lambda
        self.cf_l2loss_lambda = cf_l2loss_lambda

        self.entity_user_embed = nn.Embedding(
            self.n_entities + self.n_users, self.embed_dim
        )
        self.relation_embed = nn.Embedding(self.n_relations, self.relation_dim)
        self.trans_M = nn.Parameter(
            torch.Tensor(self.n_relations, self.embed_dim, self.relation_dim)
        )

        if (
            (self.use_pretrain == 1)
            and (user_pre_embed is not None)
            and (item_pre_embed is not None)
        ):
            other_entity_embed = nn.Parameter(
                torch.Tensor(self.n_entities - item_pre_embed.shape[0], self.embed_dim)
            )
            nn.init.xavier_uniform_(other_entity_embed)
            entity_user_embed = torch.cat(
                [item_pre_embed, other_entity_embed, user_pre_embed], dim=0
            )
            self.entity_user_embed.weight = nn.Parameter(entity_user_embed)
        else:
            nn.init.xavier_uniform_(self.entity_user_embed.weight)

        nn.init.xavier_uniform_(self.relation_embed.weight)
        nn.init.xavier_uniform_(self.trans_M)

        self.aggregator_layers = nn.ModuleList()
        for k in range(self.n_layers):
            self.aggregator_layers.append(
                Aggregator(
                    self.conv_dim_list[k],
                    self.conv_dim_list[k + 1],
                    self.mess_dropout[k],
                    self.aggregation_type,
                )
            )

        self.A_in = nn.Parameter(
            torch.sparse.FloatTensor(
                self.n_users + self.n_entities, self.n_users + self.n_entities
            )
        )
        if A_in is not None:
            self.A_in.data = A_in
        self.A_in.requires_grad = False

    def calc_cf_embeddings(self):
        ego_embed = self.entity_user_embed.weight
        all_embed = [ego_embed]

        for idx, layer in enumerate(self.aggregator_layers):
            ego_embed = layer(ego_embed, self.A_in)
            norm_embed = F.normalize(ego_embed, p=2, dim=1)
            all_embed.append(norm_embed)

        # Equation (11)
        all_embed = torch.cat(all_embed, dim=1)  # (n_users + n_entities, concat_dim)
        return all_embed

    def calc_cf_loss(self, user_ids, item_pos_ids, item_neg_ids):
        """
        user_ids:       (cf_batch_size)
        item_pos_ids:   (cf_batch_size)
        item_neg_ids:   (cf_batch_size)
        """
        all_embed = self.calc_cf_embeddings()  # (n_users + n_entities, concat_dim)
        user_embed = all_embed[user_ids]  # (cf_batch_size, concat_dim)
        item_pos_embed = all_embed[item_pos_ids]  # (cf_batch_size, concat_dim)
        item_neg_embed = all_embed[item_neg_ids]  # (cf_batch_size, concat_dim)

        # Equation (12)
        pos_score = torch.sum(user_embed * item_pos_embed, dim=1)  # (cf_batch_size)
        neg_score = torch.sum(user_embed * item_neg_embed, dim=1)  # (cf_batch_size)

        # Equation (13)
        # cf_loss = F.softplus(neg_score - pos_score)
        cf_loss = (-1.0) * F.logsigmoid(pos_score - neg_score)
        cf_loss = torch.mean(cf_loss)

        l2_loss = (
            _L2_loss_mean(user_embed)
            + _L2_loss_mean(item_pos_embed)
            + _L2_loss_mean(item_neg_embed)
        )
        loss = cf_loss + self.cf_l2loss_lambda * l2_loss
        return loss

    def calc_kg_loss(self, h, r, pos_t, neg_t):
        """
        h:      (kg_batch_size)
        r:      (kg_batch_size)
        pos_t:  (kg_batch_size)
        neg_t:  (kg_batch_size)
        """
        r_embed = self.relation_embed(r)  # (kg_batch_size, relation_dim)
        W_r = self.trans_M[r]  # (kg_batch_size, embed_dim, relation_dim)

        h_embed = self.entity_user_embed(h)  # (kg_batch_size, embed_dim)
        pos_t_embed = self.entity_user_embed(pos_t)  # (kg_batch_size, embed_dim)
        neg_t_embed = self.entity_user_embed(neg_t)  # (kg_batch_size, embed_dim)

        r_mul_h = torch.bmm(h_embed.unsqueeze(1), W_r).squeeze(
            1
        )  # (kg_batch_size, relation_dim)
        r_mul_pos_t = torch.bmm(pos_t_embed.unsqueeze(1), W_r).squeeze(
            1
        )  # (kg_batch_size, relation_dim)
        r_mul_neg_t = torch.bmm(neg_t_embed.unsqueeze(1), W_r).squeeze(
            1
        )  # (kg_batch_size, relation_dim)

        # Equation (1)
        pos_score = torch.sum(
            torch.pow(r_mul_h + r_embed - r_mul_pos_t, 2), dim=1
        )  # (kg_batch_size)
        neg_score = torch.sum(
            torch.pow(r_mul_h + r_embed - r_mul_neg_t, 2), dim=1
        )  # (kg_batch_size)

        # Equation (2)
        # kg_loss = F.softplus(pos_score - neg_score)
        kg_loss = (-1.0) * F.logsigmoid(neg_score - pos_score)
        kg_loss = torch.mean(kg_loss)

        l2_loss = (
            _L2_loss_mean(r_mul_h)
            + _L2_loss_mean(r_embed)
            + _L2_loss_mean(r_mul_pos_t)
            + _L2_loss_mean(r_mul_neg_t)
        )
        loss = kg_loss + self.kg_l2loss_lambda * l2_loss
        return loss

    def update_attention_batch(self, h_list, t_list, r_idx):
        r_embed = self.relation_embed.weight[r_idx]
        W_r = self.trans_M[r_idx]

        h_embed = self.entity_user_embed.weight[h_list]
        t_embed = self.entity_user_embed.weight[t_list]

        # Equation (4)
        r_mul_h = torch.matmul(h_embed, W_r)
        r_mul_t = torch.matmul(t_embed, W_r)
        v_list = torch.sum(r_mul_t * torch.tanh(r_mul_h + r_embed), dim=1)
        return v_list

    def update_attention(self, h_list, t_list, r_list, relations):
        device = self.A_in.device

        rows = []
        cols = []
        values = []

        for r_idx in relations:
            index_list = torch.where(r_list == r_idx)
            batch_h_list = h_list[index_list]
            batch_t_list = t_list[index_list]

            batch_v_list = self.update_attention_batch(
                batch_h_list, batch_t_list, r_idx
            )
            rows.append(batch_h_list)
            cols.append(batch_t_list)
            values.append(batch_v_list)

        rows = torch.cat(rows)
        cols = torch.cat(cols)
        values = torch.cat(values)

        indices = torch.stack([rows, cols])
        shape = self.A_in.shape
        A_in = torch.sparse.FloatTensor(indices, values, torch.Size(shape))

        # Equation (5)
        A_in = torch.sparse.softmax(A_in.cpu(), dim=1)
        self.A_in.data = A_in.to(device)

    def calc_score(self, user_ids, item_ids):
        """
        user_ids:  (n_users)
        item_ids:  (n_items)
        """
        all_embed = self.calc_cf_embeddings()  # (n_users + n_entities, concat_dim)
        user_embed = all_embed[user_ids]  # (n_users, concat_dim)
        item_embed = all_embed[item_ids]  # (n_items, concat_dim)

        # Equation (12)
        cf_score = torch.matmul(
            user_embed, item_embed.transpose(0, 1)
        )  # (n_users, n_items)
        return cf_score

    def forward(self, *input, mode):
        if mode == "train_cf":
            return self.calc_cf_loss(*input)
        if mode == "train_kg":
            return self.calc_kg_loss(*input)
        if mode == "update_att":
            return self.update_attention(*input)
        if mode == "predict":
            return self.calc_score(*input)

In [ ]:
def load_model(model, model_path):
    checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    return model

In [ ]:
# === BƯỚC 1: Chuyển đổi PySpark DataFrame thành Dictionary ===

def create_user_dict_from_spark_df(spark_df, n_entities=16118):
    """
    Chuyển đổi PySpark DataFrame thành user dictionary với ánh xạ user_id
    DataFrame cần có cột: user_id, item_id
    user_id gốc sẽ được ánh xạ thành: user_id + n_entities (mapped_user_id)
    """

    # Tạo dictionary: {mapped_user_id: [list_of_items]}
    user_dict = dict()
    for row in spark_df.rdd.collect():
        tmp = row[0].strip()
        inter = [int(i) for i in tmp.split()]
        orig_user_id, item_ids = inter[0], inter[1:]

        mapped_user_id = orig_user_id + n_entities  # Ánh xạ như trong code gốc
        item_ids = list(set(item_ids))

        user_dict[mapped_user_id] = item_ids

    return user_dict

# Sử dụng:
train_user_dict = create_user_dict_from_spark_df(train)
test_user_dict = create_user_dict_from_spark_df(test)

# === BƯỚC 2: Broadcast dictionary để sử dụng trong UDF ===

# Broadcast các dictionary
bc_train_user_dict = spark.sparkContext.broadcast(train_user_dict)
bc_test_user_dict = spark.sparkContext.broadcast(test_user_dict)

# === BƯỚC 3: Tạo Pandas UDF cho prediction ===

@pandas_udf(returnType=ArrayType(IntegerType()))
def predict_recommendations_udf(user_batch_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    """
    Pandas UDF để dự đoán recommendation cho từng batch user
    Input: mapped_user_id (user_id + n_entities)
    Output: top-K item recommendations
    """
    # Khởi tạo model (chỉ làm một lần cho mỗi executor)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    n_users = 70000
    n_entities = 16118
    n_relations = 10
    model = KGAT(
        n_users=n_users, n_entities=n_entities, n_relations=n_relations, use_pretrain=2
    )
    pretrain_model_path = "/kaggle/input/model-first-50-epochs/model_epoch100_new.pth"
    model = load_model(model, pretrain_model_path)
    model.to(device)
    model.eval()

    n_items = 3148
    item_ids = torch.arange(n_items, dtype=torch.long).to(device)

    for user_batch in user_batch_iter:
        # user_batch chứa mapped_user_id
        batch_user_ids = torch.LongTensor(user_batch.tolist()).to(device)

        with torch.no_grad():
            batch_scores = model(batch_user_ids, item_ids, mode="predict")

        # Chuyển về numpy
        batch_scores_np = batch_scores.cpu().numpy()

        # Lấy top-K items cho từng user
        TOP_K = 100
        recommendations = []

        for i in range(len(user_batch)):
            user_scores = batch_scores_np[i]
            top_items = user_scores.argsort()[::-1][:TOP_K]  # indices = item_id gốc
            recommendations.append(top_items.tolist())

        yield pd.Series(recommendations)

# === HÀM TÍNH METRICS ===
def calc_metrics_at_k(batch_scores, train_user_dict, test_user_dict, batch_user_ids, item_ids, Ks):
    """
    Tính metrics cho batch users (tương tự như code gốc của bạn)
    """
    batch_metrics = {k: {'precision': [], 'recall': [], 'ndcg': []} for k in Ks}

    for i, mapped_user_id in enumerate(batch_user_ids):
        user_scores = batch_scores[i]

        # Lấy ground truth items
        test_items = set(test_user_dict.get(mapped_user_id, []))
        train_items = set(train_user_dict.get(mapped_user_id, []))

        # Remove train items từ scores (mask với -inf)
        user_scores_filtered = user_scores.copy()
        for train_item in train_items:
            if train_item < len(user_scores_filtered):
                user_scores_filtered[train_item] = -np.inf

        # Lấy top-K items
        top_items_all = user_scores_filtered.argsort()[::-1]

        for k in Ks:
            top_k_items = set(top_items_all[:k])

            # Precision@K
            hits = len(top_k_items & test_items)
            precision = hits / k if k > 0 else 0.0

            # Recall@K
            recall = hits / len(test_items) if len(test_items) > 0 else 0.0

            # NDCG@K
            dcg = sum([1.0 / np.log2(idx + 2) for idx, item in enumerate(top_items_all[:k]) if item in test_items])
            idcg = sum([1.0 / np.log2(idx + 2) for idx in range(min(len(test_items), k))])
            ndcg = dcg / idcg if idcg > 0 else 0.0

            batch_metrics[k]['precision'].append(precision)
            batch_metrics[k]['recall'].append(recall)
            batch_metrics[k]['ndcg'].append(ndcg)

    # Convert to numpy arrays
    for k in Ks:
        for metric in ['precision', 'recall', 'ndcg']:
            batch_metrics[k][metric] = np.array(batch_metrics[k][metric])

    return batch_metrics

# === BƯỚC 4: Sử dụng UDF trên DataFrame ===

# Giả sử bạn có DataFrame chứa user_ids cần predict
users_to_predict_df = spark.createDataFrame(
    [(user_id,) for user_id in test_user_dict.keys()],
    ["user_id"]
)

# Áp dụng UDF
result_df = users_to_predict_df.select(
    col("user_id"),
    predict_recommendations_udf(col("user_id")).alias("recommendations")
)

## 1. Merge teacher, field, concept, school course file

### Visualize

In [ ]:
course_teacher = spark.read.options(header=True, inferSchema=True).text("/content/drive/MyDrive/Big Data/Output/course_teacher.txt")
course_teacher.show(5)

+-----------+
|      value|
+-----------+
| 1158 0 525|
| 1158 0 493|
| 1159 0 542|
| 1160 0 728|
|1161 0 1266|
+-----------+
only showing top 5 rows



In [ ]:
course_field = spark.read.options(header=True, inferSchema=True).text('/content/drive/MyDrive/Big Data/Output/course_field.txt')
course_field.show(5)

+-----------+
|      value|
+-----------+
|1158 1 1894|
|1158 1 1913|
|1158 1 1919|
|1158 1 1921|
|1159 1 1911|
+-----------+
only showing top 5 rows



In [ ]:
course_concept = spark.read.options(header=True, inferSchema=True).text('/content/drive/MyDrive/Big Data/Output/course_concept.txt')
course_concept.show(5)

+-----------+
|      value|
+-----------+
|1795 2 1925|
|2343 2 1925|
|1703 2 1925|
|1794 2 1925|
|2711 2 1925|
+-----------+
only showing top 5 rows



In [ ]:
course_school = spark.read.options(header=True, inferSchema=True).text('/content/drive/MyDrive/Big Data/Output/course_school.txt')
course_school.show(5)

+------------+
|       value|
+------------+
|1158 3 15941|
|1159 3 15941|
|1160 3 15941|
|1161 3 16079|
|1162 3 15941|
+------------+
only showing top 5 rows



In [ ]:
all_df = course_field.union(course_teacher).union(course_concept).union(course_school)
all_df.show(5)

+-----------+
|      value|
+-----------+
|1158 1 1894|
|1158 1 1913|
|1158 1 1919|
|1158 1 1921|
|1159 1 1911|
+-----------+
only showing top 5 rows



In [ ]:
print(f"Size of course_teacher: {course_teacher.count()}")
print(f"Size of course_field: {course_field.count()}")
print(f"Size of course_concept: {course_concept.count()}")
print(f"Size of course_school: {course_school.count()}")
print(f"Size of course_teacher + course_field + course_concept + course_school: {all_df.count()}")

Size of course_teacher: 11082
Size of course_field: 498
Size of course_concept: 93264
Size of course_school: 2658
Size of course_teacher + course_field + course_concept + course_school: 107502


In [ ]:
all_df.select("value").coalesce(1).write.mode("overwrite").text("course_all")

### Filter

In [ ]:
triplets = spark.read.options(header=True, inferSchema=True).text('/content/drive/MyDrive/Big Data/Output/kg_initial.txt')
triplets.show(5)

+-----------+
|      value|
+-----------+
|1158 1 1894|
|1158 1 1913|
|1158 1 1919|
|1158 1 1921|
|1159 1 1911|
+-----------+
only showing top 5 rows



In [ ]:
triplets = triplets.withColumn('h', split('value', ' ')[0]) \
                    .withColumn('r', split('value', ' ')[1]) \
                    .withColumn('t', split('value', ' ')[2]) \
                    .drop(col('value'))
triplets.show(5)

+----+---+----+
|   h|  r|   t|
+----+---+----+
|1158|  1|1894|
|1158|  1|1913|
|1158|  1|1919|
|1158|  1|1921|
|1159|  1|1911|
+----+---+----+
only showing top 5 rows



In [ ]:
course_type = {
    0: 'course.teacher',
    1: 'course.field',
    2: 'course.concept',
    3: 'course.school'
}

course_type_df = spark.createDataFrame(
    [(k, v) for k, v in course_type.items()],
    ["r", "relation_name"]
)

course_type_df.show()

+---+--------------+
|  r| relation_name|
+---+--------------+
|  0|course.teacher|
|  1|  course.field|
|  2|course.concept|
|  3| course.school|
+---+--------------+



In [ ]:
def filter_invalid_relations_and_entities(triplets, min_entities=5, min_rel=25):
    old_size = -1
    current_size = triplets.count()
    print(f"Old size: {current_size}")
    print("================ Before Filtering ===============")
    triplets.groupBy("r").count() \
            .join(course_type_df, on="r", how="left") \
            .select("relation_name", "count") \
            .orderBy(col("count").desc()) \
            .show(truncate=False)

    print("===== # of attribute type in each relation =====")
    # Sử dụng window function
    triplets.groupBy("r").agg(
        countDistinct("t").alias("unique_t_count")
    ).join(course_type_df, on="r", how="left") \
     .select("relation_name", "unique_t_count") \
     .orderBy("unique_t_count", ascending=False) \
     .show(truncate=False)

    while old_size != current_size:
        old_size = current_size

        # Filter entities -
        entity_counts = triplets.groupBy("t").count()
        valid_entities = entity_counts.filter(col("count") >= min_entities).select("t")
        triplets = triplets.join(valid_entities, on="t", how="inner")

        # Filter relations
        rel_counts = triplets.groupBy("r").count()
        valid_rels = rel_counts.filter(col("count") >= min_rel).select("r")
        triplets = triplets.join(valid_rels, on="r", how="inner")

        current_size = triplets.count()
        print(f"New size: {current_size}")

    # In thông tin thống kê
    print("================ Valid relations ===============")
    triplets.groupBy("r").count() \
            .join(course_type_df, on="r", how="left") \
            .select("relation_name", "count") \
            .orderBy(col("count").desc()) \
            .show(truncate=False)

    print("===== # of attribute type in each relation =====")
    # Sử dụng window function
    triplets.groupBy("r").agg(
        countDistinct("t").alias("unique_t_count")
    ).join(course_type_df, on="r", how="left") \
     .select("relation_name", "unique_t_count") \
     .orderBy("unique_t_count", ascending=False) \
     .show(truncate=False)

    return triplets

In [ ]:
fil_triplets = filter_invalid_relations_and_entities(triplets)

Old size: 107502
================ Before Filtering ===============
+--------------+-----+
|relation_name |count|
+--------------+-----+
|course.concept|93264|
|course.teacher|11082|
|course.school |2658 |
|course.field  |498  |
+--------------+-----+

===== # of attribute type in each relation =====
+--------------+--------------+
|relation_name |unique_t_count|
+--------------+--------------+
|course.concept|14016         |
|course.teacher|1862          |
|course.school |178           |
|course.field  |43            |
+--------------+--------------+

New size: 84314
New size: 84314
================ Valid relations ===============
+--------------+-----+
|relation_name |count|
+--------------+-----+
|course.concept|72220|
|course.teacher|9019 |
|course.school |2577 |
|course.field  |498  |
+--------------+-----+

===== # of attribute type in each relation =====
+--------------+--------------+
|relation_name |unique_t_count|
+--------------+--------------+
|course.concept|8163          |

In [ ]:
fil_triplets.show(5)

+---+----+----+
|  r|   t|   h|
+---+----+----+
|  1|1894|1158|
|  1|1913|1158|
|  1|1919|1158|
|  1|1921|1158|
|  1|1911|1159|
+---+----+----+
only showing top 5 rows



In [ ]:
fil_triplets = fil_triplets.select(['h', 'r', 't'])
fil_triplets.coalesce(1).write \
    .option("header", True) \
    .mode("overwrite") \
    .csv("fil_triplets")

### 2. Split user_course data

In [ ]:
user_course = spark.read.options(header=True, inferSchema=True).csv('/content/drive/MyDrive/Big Data/Output/user_course.csv')
user_course.show(truncate=False)

+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user |course_order                                                                                                                                                                                                                                                                                                                    |
+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0    |2345,1

In [ ]:
user_course = user_course.select(
    col("user").cast("int"),
    split(col("course_order"), ",").cast("array<int>").alias("course_order")
)
user_course.show(truncate=False)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user |course_order                                                                                                                                                                                                                                                                                                                                                                                       |
+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
user_course.printSchema()

root
 |-- user: integer (nullable = true)
 |-- course_order: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [ ]:
# Split data
TRAIN_PATH = 'train_data'
VAL_PATH = 'val_data'
TEST_PATH = 'test_data'

In [ ]:
def split_course_data(df, output_path="./"):
    """
    Chia DataFrame thành train, validation, test

    Args:
        df: PySpark DataFrame với cột 'user' và 'course_order'
        output_path: Đường dẫn lưu file output
    """

    # Tạo DataFrame cho test set (course cuối cùng)
    test_df = df.select(
        col("user"),
        element_at(col("course_order"), -1).alias("course_id")
    ).filter(col("course_id").isNotNull())

    # Tạo DataFrame cho validation set (course kề cuối)
    val_df = df.select(
        col("user"),
        when(size(col("course_order")) >= 2,
             element_at(col("course_order"), -2)).alias("course_id")
    ).filter(col("course_id").isNotNull())

    # Tạo DataFrame cho training set (các course còn lại)
    # Lấy tất cả course trừ 2 course cuối
    train_df = df.select(
        col("user"),
        when(size(col("course_order")) > 2,
             slice(col("course_order"), 1, size(col("course_order")) - 2)).alias("course_list")
    ).filter(col("course_list").isNotNull() & (size(col("course_list")) > 0))
    return train_df, val_df, test_df

In [ ]:
train_df, val_df, test_df = split_course_data(user_course)

In [ ]:
train_df.show(truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user |course_list                                                                                                                                                                                                                                                                                                                                                                             |
+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
val_df.show(truncate=False)

+-----+---------+
|user |course_id|
+-----+---------+
|0    |578      |
|10   |1241     |
|100  |2339     |
|1000 |2788     |
|10000|2976     |
|10003|1042     |
|10005|948      |
|10006|1077     |
|10007|1077     |
|10008|370      |
|10009|929      |
|1001 |2664     |
|10011|2361     |
|10013|538      |
|10014|573      |
|10016|1706     |
|10017|1860     |
|10018|129      |
|10019|1115     |
|10020|2931     |
+-----+---------+
only showing top 20 rows



In [ ]:
test_df.show(truncate=False)

+-----+---------+
|user |course_id|
+-----+---------+
|0    |184      |
|10   |2338     |
|100  |1348     |
|1000 |1786     |
|10000|3116     |
|10003|817      |
|10005|2670     |
|10006|2783     |
|10007|1010     |
|10008|1086     |
|10009|1038     |
|1001 |2200     |
|10011|11       |
|10013|2667     |
|10014|1058     |
|10016|2481     |
|10017|2174     |
|10018|761      |
|10019|692      |
|10020|2776     |
+-----+---------+
only showing top 20 rows



In [ ]:
# Save files
train_df.select(concat_ws(" ", col("user"), col("course_list"))).coalesce(1) \
        .write.mode("overwrite").text(TRAIN_PATH)

val_df.select(concat_ws(" ", col("user"), col("course_id"))).coalesce(1) \
      .write.mode("overwrite").text(VAL_PATH)

test_df.select(concat_ws(" ", col("user"), col("course_id"))).coalesce(1) \
       .write.mode("overwrite").text(TEST_PATH)